# Initializations

In [ ]:
!pip install langchain python-magic unstructured chromadb openai

In [ ]:
import os
import nltk
import json
import magic
import random
import datetime
import pandas as pd
from datetime import timedelta
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

!rm -rf './data'
os.makedirs('data')

os.environ["OPENAI_API_KEY"] = "HERE IS THE API KEY"

def genRandomDate():
  end_date = datetime.datetime.now()
  start_date = end_date - timedelta(days=1000)
  return str(start_date + (end_date - start_date) * random.random())

def genRandomUser():
  users = ["Poppyhead", "Zygomatic", "Calathus", "Skedaddle", "Negative", "Rejigger", "Apolaustic", "Phalanx", "Havelock", "Sardoodledom", "Galimatias", "Palomino", "Thaumatogeny", "Piffling", "Vaniloquence", "Trencherman", "Pilgarlic", "Curlicue", "KilimZol", "EnchanMizzen", "Ludicrism", "DartmoorYawp", "Anorchous", "Umpteen", "IquidDuck", "JcavalJunket", "Symptosis", "RuffminOxter", "Luciform", "Antimacassar", "Onomatopoeia", "Blackguard", "Xerotripsis", "Orgulous", "Almacantar", "Comeuppance", "ChichimWonky", "Logorrhea", "Knickknack", "Cornucopia", "Rupellary", "Peewee", "Armigerous", "Gardyloo", "Nephralgia", "Piccadilly", "Aleeman171999", "Harebrained", "Nektonseph2004", "Limburger", "Zonulet", "Sambur", "Aasvogel", "Primp", "Cinemuck", "Jubilee", "Capripede", "Flophouse", "Tachyphrasia", "Ephemeral", "Paraenesis", "Normalcy", "Morwolf1596", "Hotbed", "Limnguinhd", "Parapet", "Ateknia", "Taradiddle", "Blowback", "Kudzu", "Opsiometer", "Outflow", "Prismatic", "Roundabout", "Bellycheer", "Ewer", "Rabblefalnd", "Mufti", "Anemograph", "Cheeky"]
  return random.choice(users)


# Preparing the data

In [ ]:
data = pd.read_csv("test.csv", delimiter=",")

with open(f'data/file.jsonl', 'w', encoding='utf-8') as f:
  for row in data['review']:
    f.write(json.dumps(str({
        'username': genRandomUser(),
        'commented_on': genRandomDate(),
        'comment': row.split('READ MORE')[0]
    })) + '\n')

# Starting the main code

In [ ]:
# load all documents
loader = DirectoryLoader('data', glob="**/*.jsonl")
documents = loader.load()

# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 0,
    length_function = len,
)

texts = text_splitter.split_documents(documents)

# embed all those chunks and store them into a vectorDB (chroma)
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = Chroma.from_documents(texts, embeddings)

# initialize the QA model
qa = RetrievalQA.from_llm(llm=ChatOpenAI(), retriever=docsearch.as_retriever(), return_source_documents=True)

Exiting: Cleaning up .chroma directory
Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin


In [ ]:
while True: 
  query = input("Ques: ")
  if query == "exit": break

  response = qa({"query": query})
  display(Markdown(f"Ans: <b>{response['result']}</b>"))
  # print('source_documents', response['source_documents'])

Ques: Comments count on 2020-08-03?


DEBUG:Chroma:time to pre process our knn query: 2.384185791015625e-06
DEBUG:Chroma:time to run knn query: 0.0007333755493164062


Ans: <b>There is no comment provided in the given context on 2020-08-03, so it is not possible to determine the comment count on that date.</b>

Ques: what's on your context?


DEBUG:Chroma:time to pre process our knn query: 2.6226043701171875e-06
DEBUG:Chroma:time to run knn query: 0.0002694129943847656


...

Ans: <b>I'm sorry, I cannot give you a definitive answer without additional information as each comment has different opinions and criteria for defining the "best" headphones. It ultimately depends on your personal preferences and needs. Please let me know if you have any specific requirements or priorities, and I can try to recommend headphones that might fit your needs.</b>

Ques: Which headphone reviews are mostly positive?


DEBUG:Chroma:time to pre process our knn query: 2.86102294921875e-06
DEBUG:Chroma:time to run knn query: 0.0007929801940917969


Ans: <b>The following headphone reviews are mostly positive:

- "Nice one..battery back up is great, can be played up to 6hr if it is charged fully bass too is great..but the headphone design n build quality is not much good doesn't look so beautiful n attracting, anyway sound, bass, battery backup is much great, well n satisfaction.GO FOR IT."
- "This is my honest review after 10 months of use. I bought this for 1499 from Flipkart. If you are a bass lover go for it without any confusion. The bass is very punchy you can feel your ear vibrate with this Pros:-* Punchy bass. Don't know if any other headsets can provide this much bass* Mids & treble are crystal clear. You can hear every instrument* Battery backup is excellent."
- "excellent sound quality with superior bass backed with great battery power and feather light built quality... Mind blowing purchase.. good for music enthusiasts.. and wise choice purchase this at this price range... I'm totally loving it!!"
- "Good headphones give u decent bass. very comfortable...if u want good headphones with overall decent quality go for it."
- "even though it is little tight on your head it provides super bass and clarity. very good backup. overall fabulous product." 
- "okk i have been using these for almost 5 months now and here is my review for these headphones:- Superb sound quality, Bass is osm, minimal design, a little tight for sometime but adjusts after a week or so..,minimal controler keys, soft cushions, nice battery backup/decent charging speed, I didnt find the mic that good but overall its a beast in this price range."</b>

Ques: exit
